In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_wudao_jiewu=pd.DataFrame(columns=columns)
channel_wudao_jiewu

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [3]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [4]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [6]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').click()  #选择“舞蹈”频道

In [5]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[5]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'舞蹈'

In [6]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择“街舞”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [10]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_wudao_jiewu.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-21 07:49:07.925044


<ipython-input-10-053e3b53dc8c>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-053e3b53dc8c>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-10-053e3b53dc8c>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1LJ411n75C视频属性的抓取
BV号为1LJ411n75C的视频属性抓取结束


<ipython-input-10-053e3b53dc8c>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1G7411r7iv视频属性的抓取
BV号为1G7411r7iv的视频属性抓取结束
开始BV号为1e741167hs视频属性的抓取
BV号为1e741167hs的视频属性抓取结束
开始BV号为1VJ411n7G6视频属性的抓取
BV号为1VJ411n7G6的视频属性抓取结束
开始BV号为1V7411v7Yj视频属性的抓取
BV号为1V7411v7Yj的视频属性抓取结束
开始BV号为1v7411k7yF视频属性的抓取
BV号为1v7411k7yF的视频属性抓取结束
开始BV号为1aJ41157C6视频属性的抓取
BV号为1aJ41157C6的视频属性抓取结束
开始BV号为1aJ41157RE视频属性的抓取
BV号为1aJ41157RE的视频属性抓取结束
开始BV号为1V7411k7yE视频属性的抓取
BV号为1V7411k7yE的视频属性抓取结束
开始BV号为1rJ41157WJ视频属性的抓取
BV号为1rJ41157WJ的视频属性抓取结束
开始BV号为1TJ411J71Z视频属性的抓取
BV号为1TJ411J71Z的视频属性抓取结束
开始BV号为11J411J7A2视频属性的抓取
BV号为11J411J7A2的视频属性抓取结束
开始BV号为1L7411q7hk视频属性的抓取
BV号为1L7411q7hk的视频属性抓取结束
开始BV号为1MJ411s7UH视频属性的抓取
BV号为1MJ411s7UH的视频属性抓取结束
开始BV号为1dJ411j7XH视频属性的抓取
BV号为1dJ411j7XH的视频属性抓取结束
开始BV号为1Y7411q7GF视频属性的抓取
BV号为1Y7411q7GF的视频属性抓取结束
开始BV号为1KJ411J7jR视频属性的抓取
BV号为1KJ411J7jR的视频属性抓取结束
开始BV号为1RJ41177Qf视频属性的抓取
BV号为1RJ41177Qf的视频属性抓取结束
开始BV号为1W7411i7fq视频属性的抓取
BV号为1W7411i7fq的视频属性抓取结束
开始BV号为1S7411v7zo视频属性的抓取
BV号为1S7411v7zo的视频属性抓取结束
开始BV号为12J411j7UM视频属性的抓取
BV号为12J411j7UM的视频属性抓取结束
开始BV号为19J411H7dJ视频属性的抓取
BV号为19J411H7dJ的视

BV号为16741157WY的视频属性抓取结束
开始BV号为1o7411b7GJ视频属性的抓取
BV号为1o7411b7GJ的视频属性抓取结束
开始BV号为1U7411A75X视频属性的抓取
BV号为1U7411A75X的视频属性抓取结束
开始BV号为127411J77L视频属性的抓取
BV号为127411J77L的视频属性抓取结束
开始BV号为1F7411t7ZQ视频属性的抓取
BV号为1F7411t7ZQ的视频属性抓取结束
开始BV号为1x7411p7K1视频属性的抓取
BV号为1x7411p7K1的视频属性抓取结束
开始BV号为1f7411s7nG视频属性的抓取
BV号为1f7411s7nG的视频属性抓取结束
开始BV号为1h7411J7Gs视频属性的抓取
BV号为1h7411J7Gs的视频属性抓取结束
开始BV号为1zj411f7bw视频属性的抓取
BV号为1zj411f7bw的视频属性抓取结束
开始BV号为1S7411j7mR视频属性的抓取
BV号为1S7411j7mR的视频属性抓取结束
开始BV号为1u7411g75U视频属性的抓取
BV号为1u7411g75U的视频属性抓取结束
开始BV号为1g7411g7dv视频属性的抓取
BV号为1g7411g7dv的视频属性抓取结束
开始BV号为1b7411u7R7视频属性的抓取
BV号为1b7411u7R7的视频属性抓取结束
开始BV号为1m7411n7UR视频属性的抓取
BV号为1m7411n7UR的视频属性抓取结束
开始BV号为157411H7Bs视频属性的抓取
BV号为157411H7Bs的视频属性抓取结束
开始BV号为1i7411M7Kx视频属性的抓取
BV号为1i7411M7Kx的视频属性抓取结束
开始BV号为157411w7GG视频属性的抓取
BV号为157411w7GG的视频属性抓取结束
开始BV号为1E7411A7Sy视频属性的抓取
BV号为1E7411A7Sy的视频属性抓取结束
开始BV号为1m7411w7xs视频属性的抓取
BV号为1m7411w7xs的视频属性抓取结束
开始BV号为1o7411p7W3视频属性的抓取
BV号为1o7411p7W3的视频属性抓取结束
开始BV号为1K741187xf视频属性的抓取
BV号为1K741187xf的视频属性抓取结束
开始BV号为1p7411n7hj

BV号为1xa4y1x7v9的视频属性抓取结束
开始BV号为1Jk4y1R7C5视频属性的抓取
BV号为1Jk4y1R7C5的视频属性抓取结束
开始BV号为1YV411o71V视频属性的抓取
BV号为1YV411o71V的视频属性抓取结束
开始BV号为1Wz411B7ae视频属性的抓取
BV号为1Wz411B7ae的视频属性抓取结束
开始BV号为1U5411x7yF视频属性的抓取
BV号为1U5411x7yF的视频属性抓取结束
开始BV号为1cp4y1y7m1视频属性的抓取
BV号为1cp4y1y7m1的视频属性抓取结束
开始BV号为1pQ4y1M7rF视频属性的抓取
BV号为1pQ4y1M7rF的视频属性抓取结束
开始BV号为1A5411r7Nm视频属性的抓取
BV号为1A5411r7Nm的视频属性抓取结束
开始BV号为1hg4y1z7gZ视频属性的抓取
BV号为1hg4y1z7gZ的视频属性抓取结束
开始BV号为1dz411B7iJ视频属性的抓取
BV号为1dz411B7iJ的视频属性抓取结束
开始BV号为1b54y197Ha视频属性的抓取
BV号为1b54y197Ha的视频属性抓取结束
开始BV号为1Zt4y127eR视频属性的抓取
BV号为1Zt4y127eR的视频属性抓取结束
开始BV号为1t54y197xL视频属性的抓取
BV号为1t54y197xL的视频属性抓取结束
开始BV号为1ek4y1d7HQ视频属性的抓取
BV号为1ek4y1d7HQ的视频属性抓取结束
开始BV号为1Q5411t7LH视频属性的抓取
BV号为1Q5411t7LH的视频属性抓取结束
开始BV号为1jk4y1R741视频属性的抓取
BV号为1jk4y1R741的视频属性抓取结束
开始BV号为1Bz411B7Ve视频属性的抓取
BV号为1Bz411B7Ve的视频属性抓取结束
开始BV号为1ut4y127Xa视频属性的抓取
BV号为1ut4y127Xa的视频属性抓取结束
开始BV号为1ft4y1m7ky视频属性的抓取
BV号为1ft4y1m7ky的视频属性抓取结束
开始BV号为1gi4y187dm视频属性的抓取
BV号为1gi4y187dm的视频属性抓取结束
开始BV号为1XZ4y1x72s视频属性的抓取
BV号为1XZ4y1x72s的视频属性抓取结束
开始BV号为1yp4y1C7Wp

BV号为1ez4y1X73s的视频属性抓取结束
开始BV号为1wf4y1175F视频属性的抓取
BV号为1wf4y1175F的视频属性抓取结束
开始BV号为1eZ4y1W7Uq视频属性的抓取
BV号为1eZ4y1W7Uq的视频属性抓取结束
开始BV号为1At4y197gC视频属性的抓取
BV号为1At4y197gC的视频属性抓取结束
开始BV号为1S54y1i7Ln视频属性的抓取
BV号为1S54y1i7Ln的视频属性抓取结束
开始BV号为1Bf4y1y7iF视频属性的抓取
BV号为1Bf4y1y7iF的视频属性抓取结束
开始BV号为1uv41167xo视频属性的抓取
BV号为1uv41167xo的视频属性抓取结束
开始BV号为12z411i7hy视频属性的抓取
BV号为12z411i7hy的视频属性抓取结束
开始BV号为1sk4y1z7Ae视频属性的抓取
BV号为1sk4y1z7Ae的视频属性抓取结束
开始BV号为1hp4y1U7RA视频属性的抓取
BV号为1hp4y1U7RA的视频属性抓取结束
开始BV号为1ff4y1y7GD视频属性的抓取
BV号为1ff4y1y7GD的视频属性抓取结束
开始BV号为1554y1B7j6视频属性的抓取
BV号为1554y1B7j6的视频属性抓取结束
开始BV号为19V411C7q3视频属性的抓取
BV号为19V411C7q3的视频属性抓取结束
开始BV号为1da4y1Y7na视频属性的抓取
BV号为1da4y1Y7na的视频属性抓取结束
开始BV号为11T4y1J7RU视频属性的抓取
BV号为11T4y1J7RU的视频属性抓取结束
开始BV号为1aC4y1a7TM视频属性的抓取
BV号为1aC4y1a7TM的视频属性抓取结束
开始BV号为19z411e7Qg视频属性的抓取
BV号为19z411e7Qg的视频属性抓取结束
开始BV号为1hZ4y1W7j1视频属性的抓取
BV号为1hZ4y1W7j1的视频属性抓取结束
开始BV号为1VK411p7yu视频属性的抓取
BV号为1VK411p7yu的视频属性抓取结束
开始BV号为12g4y1i7Km视频属性的抓取
BV号为12g4y1i7Km的视频属性抓取结束
开始BV号为1tz411i7Bv视频属性的抓取
BV号为1tz411i7Bv的视频属性抓取结束
开始BV号为1Dz4y197rs

BV号为1XK4y1e71k的视频属性抓取结束
开始BV号为1Hv411q75z视频属性的抓取
BV号为1Hv411q75z的视频属性抓取结束
开始BV号为1q54y1S79S视频属性的抓取
BV号为1q54y1S79S的视频属性抓取结束
开始BV号为1M54y1S7jF视频属性的抓取
BV号为1M54y1S7jF的视频属性抓取结束
开始BV号为1AA411v7LD视频属性的抓取
BV号为1AA411v7LD的视频属性抓取结束
开始BV号为11k4y1m7A9视频属性的抓取
BV号为11k4y1m7A9的视频属性抓取结束
开始BV号为1T54y1z7R1视频属性的抓取
BV号为1T54y1z7R1的视频属性抓取结束
开始BV号为1kz4y1X7LX视频属性的抓取
BV号为1kz4y1X7LX的视频属性抓取结束
开始BV号为1pi4y1V7Zp视频属性的抓取
BV号为1pi4y1V7Zp的视频属性抓取结束
开始BV号为1cg4y1i7Wv视频属性的抓取
BV号为1cg4y1i7Wv的视频属性抓取结束
开始BV号为1H54y1S71Q视频属性的抓取
BV号为1H54y1S71Q的视频属性抓取结束
开始BV号为1LV411r7q6视频属性的抓取
BV号为1LV411r7q6的视频属性抓取结束
开始BV号为1hC4y1b73s视频属性的抓取
BV号为1hC4y1b73s的视频属性抓取结束
开始BV号为1w54y1B7ue视频属性的抓取
BV号为1w54y1B7ue的视频属性抓取结束
开始BV号为1MD4y1D7cG视频属性的抓取
BV号为1MD4y1D7cG的视频属性抓取结束
开始BV号为1h5411Y7Qv视频属性的抓取
BV号为1h5411Y7Qv的视频属性抓取结束
开始BV号为1TK411M7fo视频属性的抓取
BV号为1TK411M7fo的视频属性抓取结束
开始BV号为1dA411n7Fc视频属性的抓取
BV号为1dA411n7Fc的视频属性抓取结束
开始BV号为1wV411S79E视频属性的抓取
BV号为1wV411S79E的视频属性抓取结束
开始BV号为1i5411a7LS视频属性的抓取
BV号为1i5411a7LS的视频属性抓取结束
开始BV号为1JK4y1Y7Dq视频属性的抓取
BV号为1JK4y1Y7Dq的视频属性抓取结束
开始BV号为12f4y1X7ny

BV号为1CD4y1o7Rf的视频属性抓取结束
开始BV号为1wA411J7cz视频属性的抓取
BV号为1wA411J7cz的视频属性抓取结束
开始BV号为1gy4y1y7a6视频属性的抓取
BV号为1gy4y1y7a6的视频属性抓取结束
开始BV号为1W54y1m7pr视频属性的抓取
BV号为1W54y1m7pr的视频属性抓取结束
开始BV号为1kK4y187Hk视频属性的抓取
BV号为1kK4y187Hk的视频属性抓取结束
开始BV号为1wD4y1o7nh视频属性的抓取
BV号为1wD4y1o7nh的视频属性抓取结束
开始BV号为13t4y1q7Ha视频属性的抓取
BV号为13t4y1q7Ha的视频属性抓取结束
开始BV号为1Zp4y1e7bd视频属性的抓取
BV号为1Zp4y1e7bd的视频属性抓取结束
开始BV号为1mA411E7sU视频属性的抓取
BV号为1mA411E7sU的视频属性抓取结束
开始BV号为1MK411K7oA视频属性的抓取
BV号为1MK411K7oA的视频属性抓取结束
开始BV号为1pf4y1D79a视频属性的抓取
BV号为1pf4y1D79a的视频属性抓取结束
开始BV号为1fh411X7dw视频属性的抓取
BV号为1fh411X7dw的视频属性抓取结束
开始BV号为1Hf4y1D7Lf视频属性的抓取
BV号为1Hf4y1D7Lf的视频属性抓取结束
开始BV号为1XD4y1o7J4视频属性的抓取
BV号为1XD4y1o7J4的视频属性抓取结束
开始BV号为1d54y1C7AY视频属性的抓取
BV号为1d54y1C7AY的视频属性抓取结束
开始BV号为1g5411j7wA视频属性的抓取
BV号为1g5411j7wA的视频属性抓取结束
开始BV号为1U5411b7He视频属性的抓取
BV号为1U5411b7He的视频属性抓取结束
开始BV号为1Za4y1j7i8视频属性的抓取
BV号为1Za4y1j7i8的视频属性抓取结束
开始BV号为1Jk4y117E2视频属性的抓取
BV号为1Jk4y117E2的视频属性抓取结束
开始BV号为1z54y1C7AA视频属性的抓取
BV号为1z54y1C7AA的视频属性抓取结束
开始BV号为1dh411R7mB视频属性的抓取
BV号为1dh411R7mB的视频属性抓取结束
开始BV号为11K411K7sU

In [27]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-27-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [11]:
channel_wudao_jiewu.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [12]:
channel_wudao_jiewu.to_json('channel_wudao_jiewu.json')